## WE03a-DTrees-model-fit

In this notebook we look at using ensembles of models to improve the performance of our models. We will look at the following:
* Logestic Regression
* Logestic Regression with L2 regularization
* RandomForest
* AdaBoost
* Gradiant Boosting
* XG Boosting

RandomizedSearchCV, GridSearchCV will be preformed combinely on each model

## Import necessary packages

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier 
from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error


np.random.seed(1)

## Loading the data 

In [2]:
X_train = pd.read_csv('univbank-train_X-data.csv') 
y_train = pd.read_csv('univbank-train_y-data.csv') 
X_test = pd.read_csv('univbank-test_X-data.csv') 
y_test = pd.read_csv('univbank-test_y-data.csv')

## Observing the shapes of the data
Complete observations of the data were done in data gen note book. So not doing other observations.

In [3]:
X_train.shape

(3500, 13)

In [4]:
y_train.shape

(3500, 1)

In [5]:
X_test.shape

(1500, 13)

In [6]:
y_test.shape

(1500, 1)

### Storing the results in data frames to display them at the end.

In [7]:
rmses = pd.DataFrame({"model": [], "rmse": []})

performance = pd.DataFrame({"model": [], 'Best recall': [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

## Prediction with logestic regression model (using default parameters)

In [8]:
log_reg_model = LogisticRegression()
_ = log_reg_model.fit(X_train, np.ravel(y_train))

In [9]:
test_pred = log_reg_model.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))

rmses = pd.concat([rmses, pd.DataFrame({'model':"Log Reg", 'rmse': test_rmse}, index=[0])])

print(f"Logestic regression Test RMSE: {test_rmse}")

Logestic regression Test RMSE: 0.14832396974191325


In [10]:
model_preds = log_reg_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"default logistic", 
                                                    'Best recall': [''],
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Best recall,Accuracy,Precision,Recall,F1
0,default logistic,,0.978,1.0,0.60241,0.75188


## Prediction with logestic regression model (using Random search)

In [11]:
score_measure = 'recall'
param_grid = {
    'C': np.random.uniform(0.001, 100, size=100),
    'solver': ['liblinear', 'saga'],
    'max_iter': [500, 1000, 1500]
}
random_search_logreg = RandomizedSearchCV(
    estimator=log_reg_model,
    param_distributions=param_grid,
    n_iter=100,  # Number of parameter settings to sample
    cv=5, scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)   # Number of cross-validation folds
_ = random_search_logreg.fit(X_train, np.ravel(y_train))
print(f"The best {score_measure} score is {random_search_logreg.best_score_}")
print(f"... with parameters: {random_search_logreg.best_params_}")


best_logreg_rand = random_search_logreg.best_params_

Fitting 5 folds for each of 100 candidates, totalling 500 fits
The best recall score is 0.6662790697674419
... with parameters: {'solver': 'liblinear', 'max_iter': 1000, 'C': 51.489396316718796}


In [12]:
test_pred_logreg = random_search_logreg.predict(X_test)
test_rmse_logreg = np.sqrt(mean_squared_error(y_test, test_pred_logreg))

rmses = pd.concat([rmses, pd.DataFrame({'model':"log reg rand", 'rmse': test_rmse_logreg}, index=[0])])

print(f"Logestic regression random search Test RMSE: {test_rmse_logreg}")

Logestic regression random search Test RMSE: 0.14832396974191325


In [13]:
c_matrix = confusion_matrix(y_test, test_pred_logreg)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")
performance = pd.concat([performance, pd.DataFrame({'model':"logreg rand", 
                                                    'Best recall': [random_search_logreg.best_score_],
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

Accuracy=0.9780000 Precision=1.0000000 Recall=0.6024096 F1=0.7518797


## Prediction with logestic regression model (using exhaustive grid search on Random search)

In [14]:
score_measure = "recall"

kfolds = 5
C = random_search_logreg.best_params_['C']
solver = random_search_logreg.best_params_['solver']
max_iter = random_search_logreg.best_params_['max_iter']


param_grid = {
    'C': np.arange(C-0.0001,C+2),  
    'max_iter': np.arange(max_iter-2,max_iter+2), 
    'solver': [solver]
}

grid_search_logreg = GridSearchCV(
    estimator=log_reg_model,
    param_grid=param_grid,
    cv=5, scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
    return_train_score=True)   # Number of cross-validation folds

_ = grid_search_logreg.fit(X_train, np.ravel(y_train))

print(f"The best {score_measure} score is {grid_search_logreg.best_score_}")
print(f"... with parameters: {grid_search_logreg.best_params_}")


best_logreg_rand = grid_search_logreg.best_params_

Fitting 5 folds for each of 12 candidates, totalling 60 fits
The best recall score is 0.6662790697674419
... with parameters: {'C': 51.48929631671879, 'max_iter': 998, 'solver': 'liblinear'}


In [15]:
grid_pred_logreg = grid_search_logreg.predict(X_test)
grid_rmse_logreg = np.sqrt(mean_squared_error(y_test, grid_pred_logreg))

rmses = pd.concat([rmses, pd.DataFrame({'model':"log reg grid", 'rmse': test_rmse_logreg}, index=[0])])

print(f"Logestic regression grid search Test RMSE: {test_rmse_logreg}")

Logestic regression grid search Test RMSE: 0.14832396974191325


In [16]:
c_matrix = confusion_matrix(y_test, grid_pred_logreg)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")
performance = pd.concat([performance, pd.DataFrame({'model':"logreg grid", 
                                                    'Best recall': [grid_search_logreg.best_score_],
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

Accuracy=0.9780000 Precision=1.0000000 Recall=0.6024096 F1=0.7518797


## Prediction with logestic regression model (using L2 Regularization)

In [17]:
log_reg_L2_model = LogisticRegression(penalty='l2', max_iter=2000)
_ = log_reg_L2_model.fit(X_train, np.ravel(y_train))

In [18]:
test_pred_log = log_reg_L2_model.predict(X_test)
test_rmse_log = np.sqrt(mean_squared_error(y_test, test_pred_log))

rmses = pd.concat([rmses, pd.DataFrame({'model':"L2 Log Reg", 'rmse': test_rmse_log}, index=[0])])

print(f"L2 Logestic regression Test RMSE: {test_rmse_log}")

L2 Logestic regression Test RMSE: 0.14832396974191325


In [19]:
model_preds = log_reg_L2_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"L2 logistic",
                                                    'Best recall': [''],
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])


## Prediction with logestic regression model (using L2 Regularization, on Random search)

In [20]:
score_measure = 'recall'
param_grid = {
    'C': np.random.uniform(0.001, 100, size=100),
    'solver': ['liblinear', 'saga'],
    'max_iter': [500, 1000, 1500]
}
random_search_logregl2 = RandomizedSearchCV(
    estimator=log_reg_L2_model,
    param_distributions=param_grid,
    n_iter=100,  # Number of parameter settings to sample
    cv=5, scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)   # Number of cross-validation folds
_ = random_search_logregl2.fit(X_train, np.ravel(y_train))
print(f"The best {score_measure} score is {random_search_logregl2.best_score_}")
print(f"... with parameters: {random_search_logregl2.best_params_}")


best_logreg_rand = random_search_logregl2.best_params_

Fitting 5 folds for each of 100 candidates, totalling 500 fits
The best recall score is 0.6662790697674419
... with parameters: {'solver': 'liblinear', 'max_iter': 500, 'C': 33.80299996295606}


In [21]:
test_pred_logregl2 = random_search_logregl2.predict(X_test)
test_rmse_logregl2 = np.sqrt(mean_squared_error(y_test, test_pred_logregl2))

rmses = pd.concat([rmses, pd.DataFrame({'model':"log reg l2 rand", 'rmse': test_rmse_logregl2}, index=[0])])

print(f"Logestic regression L2 random search Test RMSE: {test_rmse_logregl2}")

Logestic regression L2 random search Test RMSE: 0.14832396974191325


In [22]:
c_matrix = confusion_matrix(y_test, test_pred_logregl2)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")
performance = pd.concat([performance, pd.DataFrame({'model':"logreg L2 rand", 
                                                    'Best recall': [random_search_logregl2.best_score_],
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

Accuracy=0.9780000 Precision=1.0000000 Recall=0.6024096 F1=0.7518797


## Prediction with logestic regression model (using L2 Regularization, exhaustive grid search on Random search)

In [23]:
score_measure = "recall"

kfolds = 5
C = random_search_logregl2.best_params_['C']
solver = random_search_logregl2.best_params_['solver']
max_iter = random_search_logregl2.best_params_['max_iter']


param_grid = {
    'C': np.arange(C-0.0001,C+2),  
    'max_iter': np.arange(max_iter-2,max_iter+2), 
    'solver': [solver]
}

grid_search_logregl2 = GridSearchCV(
    estimator=log_reg_model,
    param_grid=param_grid,
    cv=5, scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
    return_train_score=True)   # Number of cross-validation folds

_ = grid_search_logregl2.fit(X_train, np.ravel(y_train))
print(f"The best {score_measure} score is {grid_search_logregl2.best_score_}")
print(f"... with parameters: {grid_search_logregl2.best_params_}")


best_logreg_rand = grid_search_logregl2.best_params_

Fitting 5 folds for each of 12 candidates, totalling 60 fits
The best recall score is 0.6662790697674419
... with parameters: {'C': 33.802899962956054, 'max_iter': 498, 'solver': 'liblinear'}


In [24]:
grid_pred_logregl2 = grid_search_logregl2.predict(X_test)
grid_rmse_logregl2 = np.sqrt(mean_squared_error(y_test, grid_pred_logregl2))

rmses = pd.concat([rmses, pd.DataFrame({'model':"log reg grid", 'rmse': grid_rmse_logregl2}, index=[0])])

print(f"Logestic regression grid search Test RMSE: {grid_rmse_logregl2}")

Logestic regression grid search Test RMSE: 0.14832396974191325


In [25]:
c_matrix = confusion_matrix(y_test, grid_pred_logregl2)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")
performance = pd.concat([performance, pd.DataFrame({'model':"logreg grid", 
                                                    'Best recall': [grid_search_logregl2.best_score_],
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

Accuracy=0.9780000 Precision=1.0000000 Recall=0.6024096 F1=0.7518797


## Prediction with Decision Tree (using default parameters)

In [26]:
dtree=DecisionTreeClassifier()

In [27]:
_ = dtree.fit(X_train, y_train)

In [28]:
test_pred_Dtreed = dtree.predict(X_test)
test_rmse_Dtreed = np.sqrt(mean_squared_error(y_test, test_pred_Dtreed))

rmses = pd.concat([rmses, pd.DataFrame({'model':"DTree", 'rmse': test_rmse_Dtreed}, index=[0])])

print(f"Descission Tree grid search Test RMSE: {test_rmse_Dtreed}")

Descission Tree grid search Test RMSE: 0.2016597794967223


In [29]:
c_matrix = confusion_matrix(y_test, test_pred_Dtreed)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")
performance = pd.concat([performance, pd.DataFrame({'model':"DTree", 
                                                    'Best recall': [''],
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

Accuracy=0.9593333 Precision=0.6279070 Recall=0.6506024 F1=0.6390533


## Prediction with Decision Tree (using grid search and score measure = 'accuracy')

In [30]:
score_measure = "accuracy"
kfolds = 5

param_grid = {
    'min_samples_split': [2,10,50,100,200],  
    'min_samples_leaf': [1,5,10,20,50],
    'min_impurity_decrease': [0.0001, 0.0005, 0.0010, 0.0020, 0.0050],
    'max_leaf_nodes': [10,25,50,100,200], 
    'max_depth': [5,10,20,30], 
    'criterion': ['entropy', 'gini'],
}

grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTreeGrid = grid_search.best_estimator_

Fitting 5 folds for each of 5000 candidates, totalling 25000 fits
The best accuracy score is 0.9791428571428572
... with parameters: {'criterion': 'gini', 'max_depth': 5, 'max_leaf_nodes': 10, 'min_impurity_decrease': 0.0001, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [31]:
test_pred_Dtree = grid_search.predict(X_test)
test_rmse_Dtree = np.sqrt(mean_squared_error(y_test, test_pred_Dtree))

rmses = pd.concat([rmses, pd.DataFrame({'model':"DTree grid", 'rmse': test_rmse_Dtree}, index=[0])])

print(f"Descission Tree grid search Test RMSE: {test_rmse_Dtree}")

Descission Tree grid search Test RMSE: 0.14832396974191325


In [32]:
c_matrix = confusion_matrix(y_test, test_pred_Dtree)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")
performance = pd.concat([performance, pd.DataFrame({'model':"DTree grid", 
                                                    'Best recall': [''],
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

Accuracy=0.9780000 Precision=1.0000000 Recall=0.6024096 F1=0.7518797


## Prediction with Decision Tree (using random search, score measure is recall)

In [33]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': [5,10,50,100,200],  
    'min_samples_leaf': [5,10,20,50, 100],
    'min_impurity_decrease': [0.0001, 0.0005, 0.0010, 0.0020, 0.0050],
    'max_leaf_nodes': [10,25,50,100,200], 
    'max_depth': [5,10,20,30], 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=1000,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTreeRand = rand_search.best_estimator_

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits
The best recall score is 0.712262156448203
... with parameters: {'min_samples_split': 100, 'min_samples_leaf': 5, 'min_impurity_decrease': 0.001, 'max_leaf_nodes': 200, 'max_depth': 30, 'criterion': 'entropy'}


In [34]:
test_pred_randDtree = rand_search.predict(X_test)
test_rmse_randDtree = np.sqrt(mean_squared_error(y_test, test_pred_randDtree))

rmses = pd.concat([rmses, pd.DataFrame({'model':"DTree rand", 'rmse': test_rmse_randDtree}, index=[0])])

print(f"Descission Tree random search Test RMSE: {test_rmse_randDtree}")

Descission Tree random search Test RMSE: 0.18439088914585774


In [35]:
c_matrix = confusion_matrix(y_test, test_pred_randDtree)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.4f} Precision={TP/(TP+FP):.4f} Recall={TP/(TP+FN):.4f} F1={2*TP/(2*TP+FP+FN):.4f}")
performance = pd.concat([performance, pd.DataFrame({'model':"Dtree rand", 
                                                    'Best recall': [rand_search.best_score_],
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

Accuracy=0.9660 Precision=0.7051 Recall=0.6627 F1=0.6832


## Prediction with Decision Tree (using exhaustive grid search on random search)

In [36]:
score_measure = "recall"

kfolds = 5
min_samples_split = rand_search.best_params_['min_samples_split']
min_samples_leaf = rand_search.best_params_['min_samples_leaf']
min_impurity_decrease = rand_search.best_params_['min_impurity_decrease']
max_leaf_nodes = rand_search.best_params_['max_leaf_nodes']
max_depth = rand_search.best_params_['max_depth']
criterion = rand_search.best_params_['criterion']

param_grid = {
    'min_samples_split': np.arange(min_samples_split-2,min_samples_split+2),  
    'min_samples_leaf': np.arange(min_samples_leaf-2,min_samples_leaf+2),
    'min_impurity_decrease': np.arange(min_impurity_decrease-0.0001, min_impurity_decrease+0.0001, 0.00005),
    'max_leaf_nodes': np.arange(max_leaf_nodes-2,max_leaf_nodes+2), 
    'max_depth': np.arange(max_depth-2,max_depth+2), 
    'criterion': [criterion]
}

dtree = DecisionTreeClassifier()
grid_search_exh = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search_exh.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search_exh.best_score_}")
print(f"... with parameters: {grid_search_exh.best_params_}")

bestRecallTreehridexh = grid_search_exh.best_estimator_

Fitting 5 folds for each of 1280 candidates, totalling 6400 fits
The best recall score is 0.712262156448203
... with parameters: {'criterion': 'entropy', 'max_depth': 28, 'max_leaf_nodes': 198, 'min_impurity_decrease': 0.0009, 'min_samples_leaf': 3, 'min_samples_split': 98}


In [37]:
test_pred_gridDtree = grid_search_exh.predict(X_test)
test_rmse_gridDtree = np.sqrt(mean_squared_error(y_test, test_pred_gridDtree))

rmses = pd.concat([rmses, pd.DataFrame({'model':"DTree exh grid", 'rmse': test_rmse_gridDtree}, index=[0])])

print(f"Descission Tree exhaustive grid search Test RMSE: {test_rmse_gridDtree}")

Descission Tree exhaustive grid search Test RMSE: 0.18439088914585774


In [38]:
c_matrix = confusion_matrix(y_test, test_pred_gridDtree)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.4f} Precision={TP/(TP+FP):.4f} Recall={TP/(TP+FN):.4f} F1={2*TP/(2*TP+FP+FN):.4f}")
performance = pd.concat([performance, pd.DataFrame({'model':"DTree exh grid", 
                                                    'Best recall': [grid_search_exh.best_score_],
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

Accuracy=0.9660 Precision=0.7051 Recall=0.6627 F1=0.6832


## Results

In [39]:
rmses

,model,rmse
0,Log Reg,0.148324
0,log reg rand,0.148324
0,log reg grid,0.148324
0,L2 Log Reg,0.148324
0,log reg l2 rand,0.148324
0,log reg grid,0.148324
0,DTree,0.201660
0,DTree grid,0.148324
0,DTree rand,0.184391
0,DTree exh grid,0.184391


In [40]:
performance

,model,Best recall,Accuracy,Precision,Recall,F1
0,default logistic,,0.978000,1.000000,0.602410,0.751880
0,logreg rand,0.666279,0.978000,1.000000,0.602410,0.751880
0,logreg grid,0.666279,0.978000,1.000000,0.602410,0.751880
0,L2 logistic,,0.978000,1.000000,0.602410,0.751880
0,logreg L2 rand,0.666279,0.978000,1.000000,0.602410,0.751880
0,logreg grid,0.666279,0.978000,1.000000,0.602410,0.751880
0,DTree,,0.959333,0.627907,0.650602,0.639053
0,DTree grid,,0.978000,1.000000,0.602410,0.751880
0,Dtree rand,0.712262,0.966000,0.705128,0.662651,0.683230
0,DTree exh grid,0.712262,0.966000,0.705128,0.662651,0.683230


## Conclusion

To evaluate the models' performance, we need to select the most appropriate metric. There are several metrics I used for this task, including RMSE (Root Mean Square Error), accuracy, precision, recall, and the F1 score. 

1. **RMSE** 
2. **Accuracy** 
3. **Precision**
4. **Recall** 
5. **F1 score** 


* Looking at the models, it seems like Logistic Regression (default, random, grid, L2, L2 random, grid again), and Decision Tree with grid have very similar performance with RMSE 0.148324, 100% accuracy, precision of approximately 60%, recall around 75%, and an F1 score of 0.75.

* Meanwhile, the default Decision Tree, Decision Tree with random search, and Decision Tree with exhaustive grid search, have a higher RMSE and lower accuracy, precision, recall, and F1 score.

* Given these metrics, it would appear that the Logistic Regression models and the Decision Tree with grid search perform the best. 

* However, the choice of the 'best' model depends on the problem at hand. To maximizie the number of correct predictions overall, then accuracy would be the best metric, and thus the Logistic Regression and Decision Tree with grid search would be the best models.

* Overall, given the similarity across the metrics, the Logistic Regression models or the Decision Tree with grid search as the best models. However, logistic regression models are simpler and often preferable for their interpretability, so unless the Decision Tree model's structure brings some additional value, one of the logistic models might be the best choice. 
